# Table of Contents
* [Instructions](#Instructions)
* [Python Import](#Python-Import)
* [Working with Homogeneous Image](#Working-with-Homogeneous-Image)
	* [Load and Preview of data](#Load-and-Preview-of-data)
	* [Retrieve or Define Data Characteristics](#Retrieve-or-Define-Data-Characteristics)
	* [Loop Over all Slices and Correct Geometry](#Loop-Over-all-Slices-and-Correct-Geometry)
		* [Truncate Data to Keep Only Object](#Truncate-Data-to-Keep-Only-Object)
		* [Test Result by Plotting a Slice](#Test-Result-by-Plotting-a-Slice)
		* [Put Back Object Inside Image](#Put-Back-Object-Inside-Image)
		* [Output Corrected Image](#Output-Corrected-Image)
* [Working with Inomogeneous Image](#Working-with-Inomogeneous-Image)
	* [Load and Preview of data](#Load-and-Preview-of-data)
	* [Retrieve or Define Data Characteristics](#Retrieve-or-Define-Data-Characteristics)
	* [Loop Over all Slices and Correct Geometry](#Loop-Over-all-Slices-and-Correct-Geometry)
		* [Truncate Data to Keep Only Object](#Truncate-Data-to-Keep-Only-Object)
		* [Test Result by Plotting a Slice](#Test-Result-by-Plotting-a-Slice)
		* [Put Back Object Inside Image](#Put-Back-Object-Inside-Image)
		* [Output Corrected Image](#Output-Corrected-Image)


# Instructions 

This notebook will load real fits cylindrical sample images with various Intensity/radius to evaluate the difference between the output given by the formula and the real data.

# Python Import

In [1]:
from plotly.offline import plot, init_notebook_mode, iplot
init_notebook_mode()
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

import numpy as np

from IPython.display import Image
from IPython.core.display import HTML

import matplotlib.pyplot as plt
%matplotlib inline

import file_handler
import geometry_correction

CSS = """
.output {
    align-items: left;
}

div.output_area {
    width: 100%;
}
"""
HTML('<style>{}</style>'.format((CSS)))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# Working with Homogeneous Image

## Load and Preview of data

In [2]:
list_files = ['data/homogeneous_image_px_intensity_2.fits',
              'data/homogeneous_image_px_intensity_4.fits',
              'data/homogeneous_image_px_intensity_6.fits',
              'data/homogeneous_image_px_intensity_8.fits']

list_intensity = []
for _file in list_files:
    [_name, _] = os.path.splitext(_file)
    _intensity = os.path.basename(_name).split('_')[-1]
    list_intensity.append(_intensity)

homogeneous_image = []
for _file in list_files:
    homogeneous_image.append(file_handler.load_fits(_file))

## Retrieve/Define Data Characteristics and Correct Geometry

In [3]:
expected_slice_array = []
pixel_intensity_array = []
for _image in homogeneous_image:

    #image size
    [width, height] = np.shape(_image)

    # defined by user
    pixel_center = 256
    radius = 200

    _projected_image = _image[0,:]
    pixel_intensity = _projected_image[pixel_center]/(2*radius)
    pixel_intensity_array.append(pixel_intensity)

    truncated_image = _image[:, pixel_center-radius:pixel_center+radius+1]
    [height, new_width] = np.shape(truncated_image)

    # work only on 1 slice
    expected_slice = []
    _h = 0 # slice index
    _slice = truncated_image[_h, :]
    for _index, _x in enumerate(np.arange(-radius, radius+1)):
        _input_data = _slice[_index]
        _coeff = geometry_correction.homogeneous_correction_factor(x=_x, radius=radius)
        _corrected_value = (_input_data * _coeff) / 2.0
        expected_slice.append(_corrected_value)
        
    expected_slice_array.append(expected_slice)


/Users/j35/git/CylindricalGeometryCorrection/notebook/geometry_correction.py:10: RuntimeWarning:

divide by zero encountered in double_scalars



## Compare with real result

In [4]:
real_output = []
real_output.append(np.ones(len(expected_slice))*400)
real_output.append(np.ones(len(expected_slice))*800)
real_output.append(np.ones(len(expected_slice))*1200)
real_output.append(np.ones(len(expected_slice))*1600)

real_vs_expected = zip(real_output, expected_slice_array)

data = []
error_array = []
for _index, [_real, _expected] in enumerate(real_vs_expected):
    error_array.append(np.abs(_real-_expected))
    trace = go.Scatter(y=np.abs(_real[1:] - _expected[1:]),
                  name="Ipx: {}".format(list_intensity[_index]))
    data.append(trace)

figure1 = go.Figure(data=data)
iplot(figure1)


## Trying to  Find Equation of Error Profile 

In [5]:
# keep only 1 side of symetrical error signal

right_side_error_array = []
error_data = []
for _index, _error in enumerate(error_array):
    _right_side_value = _error[np.int(len(_error)/2):]
    right_side_error_array.append(_right_side_value)
    trace = go.Scatter(y=_right_side_value,
                  name="Ipx: {}".format(list_intensity[_index]))
    error_data.append(trace)
    
figure2 = go.Figure(data=error_data)
iplot(figure2)

In [6]:
# working with profile #: index
index = 0

profile = right_side_error_array[index]

r1 = 200
d = np.arange(0,len(profile))
rp = 2*r1*np.sin(np.arccos(d/r1))
factor = (2*r1)/rp * (pixel_intensity_array[index]/2)

error_data = go.Scatter(y=profile,
                        name='Error')
equation_data = go.Scatter(y=factor,
                          name='Equation')
data = [equation_data, error_data]
figure3 = go.Figure(data=data)
iplot(figure3)

/Users/j35/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning:

divide by zero encountered in true_divide



## Plot Expected with Error Bars 

In [7]:
index = 2

profile = right_side_error_array[index]

r1 = 200
d = np.arange(0,len(profile))
rp = 2*r1*np.sin(np.arccos(d/r1))
factor = (2*r1)/rp * (pixel_intensity_array[index]/2)
_error = np.concatenate((np.array(factor[::-1]), np.array(factor[1:-1])), axis=0)

_expected_plot = go.Scatter(y=expected_slice_array[index][1:],
                           name='expected',
                           mode='markers',
                           error_y = dict(type='data',
                                         array=_error,
                                         visible=True))


figure4 = go.Figure(data=[_expected_plot])
iplot(figure4)

/Users/j35/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning:

divide by zero encountered in true_divide

